### Dyanmic RLSGen Stored Procedure

In [ ]:
-- drop the procedure if it already exists
if object_id('dbo.usp_dynamicrlsgen', 'p') is not null
    drop procedure dbo.usp_dynamicrlsgen;
go

create procedure dbo.usp_dynamicrlsgen
as
begin
    set nocount on;
    print N'starting dynamic rls generation...'; 

    -- 1. delete/update/insert the queue table
    begin try
        print N'synchronizing rls processing queue...'; 
        print N'deleting obsolete entries from queue...'; 
        delete q 
        from dbo.rlsprocessingqueue as q 
        left join (
            select distinct 
                isnull(parsename(r.tablename, 2), 'dbo') as schemaname, 
                parsename(r.tablename, 1) as tablename 
            from dbo.accessrule as r 
            where r.isactive = 1 
              and parsename(r.tablename, 1) is not null
        ) as source 
            on q.schemaname = source.schemaname 
            and q.tablename = source.tablename 
        where source.schemaname is null; 
        print cast(@@rowcount as varchar) + ' rows deleted from queue.';
        
        print N'updating existing entries in queue...'; 
        update q 
        set q.isprocessed = 0, 
            q.lasterror = null, 
            q.lastprocesseddate = null 
        from dbo.rlsprocessingqueue as q 
        inner join (
            select distinct 
                isnull(parsename(r.tablename, 2), 'dbo') as schemaname, 
                parsename(r.tablename, 1) as tablename 
            from dbo.accessrule as r 
            where r.isactive = 1 
              and parsename(r.tablename, 1) is not null
        ) as source 
            on q.schemaname = source.schemaname 
            and q.tablename = source.tablename 
        where q.isprocessed = 1 
           or q.lasterror is not null; 
        print cast(@@rowcount as varchar) + ' rows updated in queue.';
        
        print N'inserting new entries into queue...'; 
        insert into dbo.rlsprocessingqueue (schemaname, tablename, isprocessed, lasterror, lastprocesseddate) 
        select 
            source.schemaname, 
            source.tablename, 
            0, 
            null, 
            null 
        from (
            select distinct 
                isnull(parsename(r.tablename, 2), 'dbo') as schemaname, 
                parsename(r.tablename, 1) as tablename 
            from dbo.accessrule as r 
            where r.isactive = 1 
              and parsename(r.tablename, 1) is not null
        ) as source 
        left join dbo.rlsprocessingqueue as q 
            on q.schemaname = source.schemaname 
            and q.tablename = source.tablename 
        where q.schemaname is null; 
        print cast(@@rowcount as varchar) + ' rows inserted into queue.';
        
        declare @activetables int; 
        select @activetables = count(*) from dbo.rlsprocessingqueue where isprocessed = 0; 
        print N'queue synchronized. processing ' + cast(@activetables as varchar) + ' table(s).'; 
    end try
    begin catch
        print N'fatal error: could not synchronize dbo.rlsprocessingqueue: ' + error_message(); 
        throw;
    end catch;

    -- 2. declare variables
        declare @currentschemaname nvarchar(128); 
        declare @currenttablename nvarchar(128); 
        declare @fulltablename nvarchar(255); 
        declare @functionname nvarchar(255); 
        declare @policyname nvarchar(255);
        declare @sql nvarchar(max); 
        declare @paramlist nvarchar(max); 
        declare @collistforpolicy nvarchar(max); 
        declare @whereclause nvarchar(max);
        declare @errormessage nvarchar(max); 
        declare @objectid int;
        declare @columnlistforreplace nvarchar(max); 
        declare @modifiedrulefilterexpression nvarchar(max); 
        declare @currentcolumnname nvarchar(128);
        declare @replacepos int; 
        declare @replacenextcomma int;
        declare @fetchrolename nvarchar(128); 
        declare @fetchfilterexpression nvarchar(max);
        declare @rownum int = 0;
        declare @allfilterexpressions nvarchar(max);
        declare @paddedfilterexpressions nvarchar(max);

    -- 3a. process tables from queue
    while (select count(*) from dbo.rlsprocessingqueue where isprocessed = 0) > 0
    begin
        select top 1 
            @currentschemaname = schemaname, 
            @currenttablename = tablename 
        from dbo.rlsprocessingqueue 
        where isprocessed = 0 
        order by schemaname, tablename;
        
        set @fulltablename = quotename(@currentschemaname) + N'.' + quotename(@currenttablename); 
        set @functionname = quotename(@currentschemaname) + N'.' + quotename(N'fn_predicate_' + @currenttablename); 
        set @policyname = quotename(@currentschemaname) + N'.' + quotename(@currenttablename + N'_rls_policy');
        set @errormessage = null; 
        set @objectid = object_id(@fulltablename);
        print char(13) + char(10) + N'--- processing: schema ['+ @currentschemaname + N'], table [' + @currenttablename + N'] ---'; 

        begin try
            if @objectid is null 
            begin 
                set @errormessage = N'target table/view ' + @fulltablename + N' not found.'; 
                print N'error: ' + @errormessage; 
                raiserror(@errormessage, 16, 1); 
            end; 
            print N'table object id found: ' + cast(@objectid as varchar); 

            -- 3b. identify required columns and build filtered lists
            print N'identifying required columns and generating lists...';
            set @paramlist = null; 
            set @collistforpolicy = null; 
            set @columnlistforreplace = null; 
            set @allfilterexpressions = null; 
            set @paddedfilterexpressions = null;

        -- step 3b.1: get all active filter expressions for this table
            select @allfilterexpressions = string_agg(cast(filterexpression as nvarchar(max)), N' ') 
            from dbo.accessrule
            where parsename(tablename, 1) = @currenttablename
              and isnull(parsename(tablename, 2), 'dbo') = @currentschemaname
              and isactive = 1;

            if @allfilterexpressions is null or ltrim(rtrim(@allfilterexpressions)) = N'' 
            begin
                 print N'warning: no active filter expressions found for ' + @fulltablename + N'. function will have no parameters.'; 
                 set @paddedfilterexpressions = N' '; 
            end
            else
            begin
                 -- pad with spaces for improved like check
                 set @paddedfilterexpressions = N' ' + @allfilterexpressions + N' '; 
                 print N'collected and padded filter expressions for column usage check.'; 
            end

        -- step 3b.2: generate lists using separate selects. 
            print N'generating lists.'; 

            -- first select for paramlist and collistforpolicy
            select
                -- paramlist: include only if column name is found bounded by non-identifier chars
                @paramlist = string_agg(
                    case
                        -- use the improved like check against the padded expression string
                        when @paddedfilterexpressions like N'%[^a-za-z0-9_]' + c.name + N'[^a-za-z0-9_]%' escape N'\' 
                        then cast(N'@' + c.name + N' ' + 
                            case when t.name in ('varchar','char','nvarchar','nchar') 
                                 then t.name + N'(' + iif(c.max_length = -1, 'max', cast(c.max_length as varchar(10))) + N')' 
                                 when t.name in ('decimal','numeric') 
                                 then t.name + N'(' + cast(c.precision as varchar(5)) + N',' + cast(c.scale as varchar(5)) + N')' 
                                 when t.name in ('datetime2','time','datetimeoffset') 
                                 then t.name + N'(' + cast(c.scale as varchar(5)) + N')' 
                                 else t.name 
                            end as nvarchar(max)) 
                        else null -- exclude column if pattern doesn't match
                    end, N', ') within group (order by c.column_id), 

                -- collistforpolicy: include only if column name is found bounded by non-identifier chars
                @collistforpolicy = string_agg(
                    case
                        -- use the improved like check against the padded expression string
                        when @paddedfilterexpressions like N'%[^a-za-z0-9_]' + c.name + N'[^a-za-z0-9_]%' escape N'\' 
                        then cast(quotename(c.name) as nvarchar(max))
                        else null -- exclude column if pattern doesn't match
                    end, N', ') within group (order by c.column_id) 
            from sys.columns as c 
            join sys.types as t on c.user_type_id = t.user_type_id
            where c.object_id = @objectid;

             select --second select
                 @columnlistforreplace = string_agg(cast(c.name as nvarchar(max)), N',') within group (order by len(c.name) desc, c.name) 
            from sys.columns as c
            where c.object_id = @objectid;

            -- check resulting lists
            if @paramlist is null set @paramlist = N''; 
            if @collistforpolicy is null set @collistforpolicy = N''; 

            if @paramlist = N'' 
                begin print N'warning: parameter list for function is empty (no columns detected in expressions).'; end 
            else 
                print N'parameter list generated (strictly filtered).'; 
                
            if @collistforpolicy = N'' 
                begin print N'warning: column list for policy is empty (no columns detected in expressions).'; end 
            else 
                print N'policy column list generated (strictly filtered).'; 
                
            if @columnlistforreplace is null or @columnlistforreplace = N'' 
                begin print N'warning: no columns found for replacement list.'; end 
            else 
                print N'column list for replacement generated.'; 

            -- 3c. build the where clause rule-by-rule
            print N'building where clause rule-by-rule.'; 
            set @whereclause = N''; 
            set @rownum = 0; 
            
            while 1=1 
            begin 
                select top 1 
                    @fetchrolename = rolename, 
                    @fetchfilterexpression = filterexpression 
                from (
                    select 
                        rolename, 
                        filterexpression, 
                        row_number() over(order by (select null)) as rn 
                    from dbo.accessrule 
                    where parsename(tablename, 1) = @currenttablename 
                      and isnull(parsename(tablename, 2), 'dbo') = @currentschemaname 
                      and isactive = 1
                ) as ruleswithrownum 
                where rn = @rownum + 1; 
                
                if @@rowcount = 0 break; 
                
                set @rownum = @rownum + 1; 
                set @modifiedrulefilterexpression = @fetchfilterexpression; 
                
                if @columnlistforreplace is not null and @columnlistforreplace <> N'' 
                begin 
                    set @replacepos = 1; 
                    while @replacepos <= len(@columnlistforreplace) and charindex(N',', @columnlistforreplace, @replacepos) > 0 
                    begin 
                        set @replacenextcomma = charindex(N',', @columnlistforreplace, @replacepos); 
                        set @currentcolumnname = substring(@columnlistforreplace, @replacepos, @replacenextcomma - @replacepos); 
                        if @currentcolumnname <> N'' 
                            set @modifiedrulefilterexpression = replace(@modifiedrulefilterexpression, @currentcolumnname, N'@' + @currentcolumnname); 
                        set @replacepos = @replacenextcomma + 1; 
                    end 
                    
                    if @replacepos <= len(@columnlistforreplace) 
                    begin 
                        set @currentcolumnname = substring(@columnlistforreplace, @replacepos, len(@columnlistforreplace) - @replacepos + 1); 
                        if @currentcolumnname <> N'' 
                            set @modifiedrulefilterexpression = replace(@modifiedrulefilterexpression, @currentcolumnname, N'@' + @currentcolumnname); 
                    end 
                end 
                
                if @whereclause <> N'' 
                    set @whereclause = @whereclause + nchar(13) + nchar(10) + N'    or '; 
                    
                set @whereclause = @whereclause + N'( is_member(N''' + replace(@fetchrolename, N'''', N'''''') + N''') = 1 and (' + @modifiedrulefilterexpression + N') )'; 
            end 
            
            if @whereclause is null or @whereclause = N'' 
            begin 
                print N'warning: no active rules found or processed for table ' + @fulltablename + N'. function will deny all access.'; 
                set @whereclause = N'1 = 0'; 
            end 
            else 
            begin 
                print N'where clause built successfully rule-by-rule.'; 
            end 

            -- 3d. drop existing policy and function
            print N'dropping existing policy and function if they exist...'; 

            set @sql = N'if exists (select * from sys.security_policies where name = N''' + replace(parsename(@policyname, 1), N'''', N'''''') + 
                       N''' and schema_id = schema_id(N''' + replace(@currentschemaname, N'''', N'''''') + N''')) drop security policy ' + @policyname + N';'; 
            print N'-- executing: ' + @sql; 
            exec sp_executesql @sql; 
            print N'existing policy ' + @policyname + N' dropped (if it existed).'; 
            
            set @sql = N'if object_id(N''' + replace(@functionname, N'''', N'''''') + N''', N''if'') is not null drop function ' + @functionname + N';'; 
            print N'-- executing: ' + @sql; 
            exec sp_executesql @sql; 
            print N'existing function ' + @functionname + N' dropped (if it existed).'; 

            -- 3e. create new function and policy
            print N'generating new rls function ' + @functionname + N'...'; 
            set @sql = N'create function ' + @functionname + N' (' + @paramlist + N') returns table with schemabinding as return select 1 as _predicateresult where ' +
                       @whereclause + N';'; 
            print N'-- generating function:' + nchar(13) + nchar(10) + @sql; 
            exec sp_executesql @sql;
            print N'function ' + @functionname + N' created successfully.'; 
            
            if @paramlist <> N'' and @collistforpolicy = N'' 
            begin
                 set @errormessage = N'function expects parameters based on detected columns, but the column list for policy call is empty 
                 (check filter expressions and column names). cannot create policy for ' + @fulltablename + N'.'; 
                 print N'error: ' + @errormessage; 
                 raiserror(@errormessage, 16, 1); 
            end
            
            print N'generating new security policy ' + @policyname + N' on table ' + @fulltablename + N'...'; 
            set @sql = N'create security policy ' + @policyname + N' add filter predicate ' + @functionname + N'(' + @collistforpolicy + N') on ' 
                      + @fulltablename + N' with (state = on);'; 
            print N'-- generating policy:' + nchar(13) + nchar(10) + @sql; 
            exec sp_executesql @sql;
            print N'security policy ' + @policyname + N' created successfully.'; 

            update dbo.rlsprocessingqueue 
            set isprocessed = 1, 
                lasterror = null, 
                lastprocesseddate = getdate() 
            where schemaname = @currentschemaname 
              and tablename = @currenttablename;
              
            print N'finished processing: schema ['+ @currentschemaname + N'], table [' + @currenttablename + N'].'; 
        end try
        begin catch
            set @errormessage = N'error processing schema ['+ @currentschemaname + N'], table [' + @currenttablename + N']: ' + error_message(); 
            print @errormessage;
            update dbo.rlsprocessingqueue 
            set isprocessed = 1, 
                lasterror = @errormessage, 
                lastprocesseddate = getdate() 
            where schemaname = @currentschemaname 
              and tablename = @currenttablename;
        end catch;
    end; -- end of first while loop

    print N'dynamic rls generation run complete.'; 
    set nocount off;
    return;
end;
go

print N'stored procedure [dbo.usp_dynamicrlsgen] created/updated.'; 
go
